In [1]:
#Use with python/3.6.0 and tensorflow/1.10

import math
import sys
from sklearn.linear_model import LogisticRegression
import random
import os
import csv
import json
from collections import defaultdict
import gensim
import numpy as np
import pandas as pd

from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegressionCV
import sklearn.metrics as metrics
from sklearn import metrics
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from glove import Corpus, Glove
from bert_serving.client import BertClient
bc = BertClient()
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers
 


from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout

from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import GridSearchCV, KFold

from sklearn.metrics import accuracy_score
from tensorflow.python.keras.constraints import maxnorm
from tensorflow.python.keras.callbacks import EarlyStopping
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error, mean_squared_error
import itertools
lib_path = os.path.abspath(os.path.join('..', 'lib'))
sys.path.append(lib_path)

ds = "mimic"
data_path = "/users/emily/Documents/mimic_seq/"
#data_path = "/home/egetzen/jill_mimic_seq/"
window=10
size=100
decay=5
skipgram=1
norm=False
random.seed(1)

# list for files
train_files = []
valid_files = []
full_data_files = []

In [2]:
def get_file_list(data_path):
    # list for files
    train_files = []
    valid_files = []
    full_data_files = []

    for i in range(7):
        train_files.append(data_path + 'test_'+str(i))
    for i in range(7,10):
        valid_files.append(data_path + 'test_'+str(i))
    for i in range(10):
        full_data_files.append(data_path + 'test_'+str(i))
    
    return train_files, valid_files, full_data_files

In [4]:
train_files, valid_files, full_data_files = get_file_list(data_path)

In [5]:
sentences = []
final_dx_train = []
disease_prev = []
list_duplicates = []
diag = []

In [6]:
#test = ['d_041','d_250','d_401','d_414','d_424','d_427','d_530','d_585']
test = ['d_585']

In [7]:
for q in range(len(test)):
    def create_disease_sentences(files):
        count = -1
        sentences = []
        demographic = []
        final_dx = []
        disease_prev = []

        for i in files:
            with open(i) as f:
                for s in f:
                    count = count + 1
                    demographic.append(s.split("|")[1].split(" "))
                    sentences.append(s.split("|")[2].split(" ") +
                                     s.split("|")[3].replace("\n", "").split(" "))
                    next_diags = s.split("|")[0].split(",")
                    final_dx.append(next_diags)
                    prev_diags = [e for e in s.split("|")[2].split(" ") if e.startswith("d_")]

                    if test[q] in prev_diags: # if disease exit in the previous diagnosis
     
                            disease_prev.append(1)
                    
                    else:
                        disease_prev.append(0)

        # create a vector indicating which patients have disease as a final diagnosis
        disease_final_ori = []
        for sublist in final_dx:
            if test[q] in sublist:
                disease_final_ori.append(1)
            else:
                disease_final_ori.append(0)


        return sentences, demographic, final_dx, disease_prev, disease_final_ori
    sentences, demographic, final_dx_train, disease_prev, disease_train_ori = create_disease_sentences(train_files)
    def generate_bert_dict(sentences):

        flat_sentences = list(itertools.chain(*sentences))

        events = np.unique(flat_sentences)
        events = np.ndarray.tolist(events)

        bert_dict = []
        bertdict = []

        return events, bert_dict, bertdict
    events, bert_dict, bertdict = generate_bert_dict(sentences)

    def generate_patient_disease_vectors(files, events, bert_dict):
        count = -1
        count2 = 0

        patient_seq_all_disease = []
        patient_seq_all_disease_1 = []
        patient_sen_disease = []
        exclude = []

        _, _, _, disease_prev, disease_final_ori = create_disease_sentences(files)

        for i in files: 
            with open(i) as f:
                for line in f:
                    count = count + 1
                    patient_seq = []
                    feed_events = line.split("|")[2].split(" ")

                    if disease_prev[count] == 1:
                        target = feed_events.index(test[q])
                        feed_events = feed_events[:target]
                        if feed_events == []:
                            count2 = count2+1
                            exclude.append(1)
                        else:    
                            exclude.append(0)
                            te = len(feed_events)
                            
                            patient_seq = 0
                            patient_seq_all_disease.append(patient_seq)
                            newfeed = []
                            newfeed1 = []
                            newfeed2 = []
                            patient_seq = []
                            for n in range(len(feed_events)):
                                newfeed.append(feed_events[n].replace("_",r""))
                            for y in range(len(newfeed)):
                                newfeed1.append(newfeed[y].replace(".",r""))
                            for z in range(len(newfeed1)):
                                newfeed2.append(newfeed1[z].replace("/",r"q"))
                            feed_events = newfeed2

                            feed = ' '.join(feed_events)
                            patient_seq = bc.encode([feed])
                            patient_seq_all_disease_1.append(patient_seq)

                            
                            

                    else: 
                        exclude.append(0)
                        te = len(feed_events)
                        
                        patient_seq = 0
                        patient_seq_all_disease.append(patient_seq)
                        newfeed = []
                        newfeed1 = []
                        newfeed2 = []
                        patient_seq = []
                        for n in range(len(feed_events)):
                            newfeed.append(feed_events[n].replace("_",r""))
                        for y in range(len(newfeed)):
                            newfeed1.append(newfeed[y].replace(".",r""))
                        for z in range(len(newfeed1)):
                            newfeed2.append(newfeed1[z].replace("/",r"q"))
                        feed_events = newfeed2

                        feed = ' '.join(feed_events)
                        patient_seq = bc.encode([feed])
                        patient_seq_all_disease_1.append(patient_seq)

        patient_seq_all_disease = []
        
    
        flat_embeddings = list(itertools.chain(*patient_seq_all_disease_1))

        #data_disease=pd.DataFrame(patient_seq_all_disease)
        data_disease_1=pd.DataFrame(flat_embeddings)

        disease_final = []
        for i in range(0,len(exclude)):
            if exclude[i] == 0:
                disease_final.append(disease_final_ori[i])

        #data_disease[test[q]] = disease_final
        data_disease_1[test[q]] = disease_final

       # X_disease=data_disease.iloc[:,0:100]
        X_disease_1=data_disease_1.iloc[:,0:100]
        Y_disease = data_disease_1.iloc[:,100]

        return X_disease_1, Y_disease, data_disease_1

    X_train_disease_1, Y_train_disease, data_disease_1 = generate_patient_disease_vectors(train_files, events, bert_dict)
    X_disease_test_1, Y_disease_test, data_disease_test_1 = generate_patient_disease_vectors(valid_files, events, bert_dict)
    



/opt/anaconda3/envs/[py3.6]/lib/python3.6/site-packages/bert_serving/client/__init__.py:290: UserWarning: server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficiency
  warnings.warn('server does not put a restriction on "max_seq_len", '


In [7]:
feeds

NameError: name 'feeds' is not defined

In [8]:
import openpyxl
import pandas as pd
data_disease_1.to_excel('/users/emily/Documents/sentence/ckd_train.xlsx')
data_disease_test_1.to_excel('/users/emily/Documents/sentence/ckd_test.xlsx')

In [154]:
sentenzes=[]
for n in range(len(sentences)):
    sentz = []
    for i in range(len(sentences[n])):
        sentz.append(sentences[n][i].replace("_",r""))
    sentenzes.append(sentz)

In [155]:
sentences = []
for n in range(len(sentenzes)):
    sentz = []
    for i in range(len(sentenzes[n])):
        sentz.append(sentenzes[n][i].replace(".",r""))
    sentences.append(sentz)

In [156]:
pls = []
for n in range(len(sentences)):
    sentz = []
    for i in range(len(sentences[n])):
        sentz.append(sentences[n][i].replace("/",r"q"))
    pls.append(sentz)

In [157]:
sentences = pls

In [210]:
flat_sentences = list(itertools.chain(*sentences))

events = np.unique(flat_sentences)
events = np.ndarray.tolist(events)
new = []
new_feed_events = []
vec = []
bigvec = []
newfeed = []
patient_seq = []
for n in range(len(events)):
    newfeed.append(events[n].replace("_",r"\u"))
for g in range(len(newfeed)):
    feeds = (newfeed[g])
    texts2 = [feeds]
    if feeds == 's78899':
        print('hi')
    else:
        veci = bc.encode([texts2],is_tokenized=True)
        vec.append(veci[0][1])
    
      
 




KeyboardInterrupt: 

In [209]:
bc.encode([['s78899']],is_tokenized=True)

KeyboardInterrupt: 

In [207]:
texts2

['s78899']

1.0

In [ ]:
import openpyxl
import pandas as pd
data_disease_1.to_excel('/users/emily/Documents/sentence/strep_train.xlsx')
data_disease_test_1.to_excel('/users/emily/Documents/sentence/strep_test.xlsx')

In [ ]:
import openpyxl
import pandas as pd
data_disease.to_excel('/users/emily/Documents/normal/strep_train.xlsx')
data_disease_test.to_excel('/users/emily/Documents/normal/strep_test.xlsx')

In [8]:
for q in range(len(test)):
    
    count = -1
    sentences = []
    demographic = []
    final_dx = []
    disease_prev = []
    
    for i in train_files:
        with open(i) as f:
            for s in f:
                count = count + 1
                demographic.append(s.split("|")[1].split(" "))
                sentences.append(s.split("|")[2].split(" ") +
                                 s.split("|")[3].replace("\n", "").split(" "))
                next_diags = s.split("|")[0].split(",")
                final_dx.append(next_diags)
                prev_diags = [e for e in s.split("|")[2].split(" ") if e.startswith("d_")]

                if test[q] in prev_diags: # if disease exit in the previous diagnosis

                        disease_prev.append(1)

                else:
                    disease_prev.append(0)
                    
    flat_sentences = list(itertools.chain(*sentences))

    events = np.unique(flat_sentences)
    events = np.ndarray.tolist(events)
    newfeed = []
    newfeed1 = []
    newfeed2 = []
   
    for n in range(len(events)):
        newfeed.append(events[n].replace("_",r""))
    for y in range(len(newfeed)):
        newfeed1.append(newfeed[y].replace(".",r""))
    for z in range(len(newfeed1)):
        newfeed2.append(newfeed1[z].replace("/",r"q"))
    events = newfeed2
    
    
    
    # create a vector indicating which patients have disease as a final diagnosis
    disease_final_ori = []
    for sublist in final_dx:
        if test[q] in sublist:
            disease_final_ori.append(1)
        else:
            disease_final_ori.append(0)


   
    count = -1
    count2 = 0

    
    patient_seq_all_disease_1 = []
    exclude = []

    

    for i in train_files: 
        with open(i) as f:
            for line in f:
                count = count + 1
                patient_seq = []
                feed_events = line.split("|")[2].split(" ")
                
                if disease_prev[count] == 1:
                    target = feed_events.index(test[q])
                    feed_events = feed_events[:target]
                    if feed_events == []:
                        count2 = count2+1
                        exclude.append(1)
                    else:    
                        exclude.append(0)
                        te = len(feed_events)
                        
                        newfeed = []
                        newfeed1 = []
                        newfeed2 = []
                        patient_seq = []
                        for n in range(len(feed_events)):
                            newfeed.append(feed_events[n].replace("_",r""))
                        for y in range(len(newfeed)):
                            newfeed1.append(newfeed[y].replace(".",r""))
                        for z in range(len(newfeed1)):
                            newfeed2.append(newfeed1[z].replace("/",r"q"))
                        feed_events = newfeed2
                        feed = ' '.join(feed_events)
                        patient_seq = bc.encode([feed])
                        patient_seq_all_disease_1.append(patient_seq)
                        

                else: 
                    exclude.append(0)
                    te = len(feed_events)
                    
                    newfeed = []
                    newfeed1 = []
                    newfeed2 = []
                    patient_seq = []
                    for n in range(len(feed_events)):
                        newfeed.append(feed_events[n].replace("_",r""))
                    for y in range(len(newfeed)):
                        newfeed1.append(newfeed[y].replace(".",r""))
                    for z in range(len(newfeed1)):
                        newfeed2.append(newfeed1[z].replace("/",r"q"))
                    feed_events = newfeed2
                    
                    feed = ' '.join(feed_events)
                    patient_seq = bc.encode([feed])
                    patient_seq_all_disease_1.append(patient_seq)

    flat_embeddings = list(itertools.chain(*patient_seq_all_disease_1))

    #data_disease=pd.DataFrame(patient_seq_all_disease)
    data_disease_1=pd.DataFrame(flat_embeddings)

    disease_final = []
    for i in range(0,len(exclude)):
        if exclude[i] == 0:
            disease_final.append(disease_final_ori[i])

    #data_disease[test[q]] = disease_final
    data_disease_1[test[q]] = disease_final

   # X_disease=data_disease.iloc[:,0:100]
    X_disease_1=data_disease_1.iloc[:,0:100]

    #Y_disease =data_disease.iloc[:,100]


KeyboardInterrupt: 

In [9]:
feed

'l50931 l51221 l51222 l51237 l51244 l51249 l51256 l51274 l51279 l50809 pACET325 pBISA10R pBISA5 pDOCU100L pFURO40I pFURO40I pHEPA100SYR pHEPA5I pLOMO pOXYC20 pOXYC5 pPANCEC pPROC10 pSENN187 pTHIA100 l51498 l50893 l51248 l51254 l50818 l50820 l50821 s7806 s7881 d157 d197 d197 d272 d458 d710 dE9331'

In [285]:
flat_embeddings = list(itertools.chain(*patient_seq_all_disease_1))

#data_disease=pd.DataFrame(patient_seq_all_disease)
data_disease_1=pd.DataFrame(flat_embeddings)

disease_final = []
for i in range(0,len(exclude)):
    if exclude[i] == 0:
        disease_final.append(disease_final_ori[i])

#data_disease[test[q]] = disease_final
data_disease_1[test[q]] = disease_final

# X_disease=data_disease.iloc[:,0:100]
X_disease_1=data_disease_1.iloc[:,0:100]

In [286]:
data_disease_1

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,d_250
0,-0.657563,-1.466410,-0.783210,1.806302,1.695678,-1.613397,-0.712142,0.470452,-0.133599,-0.364709,...,2.053072,-0.312814,-0.489238,-0.695473,1.280036,-0.105422,0.653839,0.472223,-0.873382,0
1,-0.774092,-1.597584,-0.908868,1.757128,1.622630,-1.651690,-0.689092,0.409494,-0.111996,-0.379726,...,2.101107,-0.374771,-0.730008,-0.597803,1.223652,-0.132777,0.560741,0.617831,-0.933828,1
2,-0.791636,-1.604244,-0.936580,1.759053,1.605657,-1.669669,-0.694756,0.421829,-0.097583,-0.397456,...,2.095240,-0.377907,-0.722901,-0.615878,1.206666,-0.119395,0.575785,0.634548,-0.934142,1
3,-0.766865,-1.482419,-0.925181,1.736856,1.608101,-1.623577,-0.658383,0.409182,-0.100596,-0.404779,...,2.125177,-0.358202,-0.622099,-0.673460,1.218296,-0.088147,0.532072,0.549934,-0.888171,1
4,-0.763086,-1.621967,-0.924811,1.745443,1.605310,-1.643287,-0.680831,0.429845,-0.087782,-0.372354,...,2.094303,-0.368139,-0.711969,-0.587385,1.200658,-0.135743,0.545149,0.622489,-0.940470,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3941,-0.802858,-1.613631,-0.943025,1.790726,1.604323,-1.657951,-0.688836,0.410952,-0.108600,-0.394783,...,2.116454,-0.403939,-0.774593,-0.597060,1.216689,-0.109273,0.561272,0.650537,-0.952175,1
3942,-0.762673,-1.550623,-0.885201,1.792213,1.600774,-1.651527,-0.667865,0.434147,-0.089151,-0.352441,...,2.082204,-0.371435,-0.686780,-0.606140,1.229220,-0.130517,0.576805,0.602121,-0.930329,1
3943,-0.775334,-1.606487,-0.928022,1.759038,1.599843,-1.650989,-0.679962,0.442453,-0.098948,-0.361156,...,2.089180,-0.367931,-0.720406,-0.576141,1.209271,-0.123384,0.561684,0.616913,-0.932948,0
3944,-0.711548,-1.564356,-0.900320,1.762469,1.601174,-1.649582,-0.685905,0.472015,-0.121987,-0.340226,...,2.008133,-0.351103,-0.662387,-0.559911,1.256652,-0.115914,0.556708,0.573569,-0.901672,0
